## FAQ task

FAQ consists of questions:
1. What is preparatory course?
 - Preparatory course is a special educational program lasting 1 academic year (7-10 months), where students learn Russian and special disciplines (mathematics and physics).
2. What is invitation letter?
 - The invitation is official document which is prepared by Ministry of Internal Affairs of Russian Federation. It confirms that the student is admitted to this university.
3. ...


Now you have questions from users and you need to answer, for example:

:: Could I work while studying?
> It allows the student to find well paid work and to start climbing up on a career ladder right after completing university course. Students of the Russian universities are obliged to attend all lectures as only the knowledge gained during classroom occupations allows students to become the effective and knowing professionals. 


In this tuorial we'll describe how to build FAQ model based on config deeppavlov/configs/faq/tfidf_logreg_en_faq.json
<br>First of all we need train dataset of FAQ. As example, let's consider MIPT FAQ for entrants - https://mipt.ru/english/edu/faqs/

**Note:** Please, install all necessary requirements using command:

>\>\> python -m deeppavlov install deeppavlov/configs/faq/tfidf_logreg_en_faq.json

Let's look at the FAQ dataset:

In [8]:
import pandas as pd
FAQ_DATASET_URL = 'http://files.deeppavlov.ai/faq/mipt/faq.csv'
faq_dataset = pd.read_csv(FAQ_DATASET_URL)
faq_dataset

,Question,Answer
0,What is preparatory course?,Preparatory course is a special educational pr...
1,What is invitation letter?,The invitation is official document which is p...
2,What is registration?,Registration grants to the foreign citizen the...
3,Is it possible to study and work at the same t...,Russian education is one of the most qualitati...
4,How long does the academic year last?,Academic year proceeds 10 months (from Septemb...
5,What documents are demanded for admission?,"Passport, documents of your previous education..."
6,What is the price for one year of study?,Russian taught programs cost 250'000 rubles pe...
7,Should I insure my life?,Life insurance and health is obligatory for an...
8,In what cases student can be deducted from Uni...,"At own will, for health reasons, for the acade..."
9,I have problems. Who can help me?,If you have any problems you can address to De...


In [9]:
import deeppavlov
from deeppavlov.models.classifiers.logreg_classifier import LogregClassifier
from deeppavlov.models.tokenizers.spacy_tokenizer import StreamSpacyTokenizer
from deeppavlov.dataset_readers.faq_reader import FaqDatasetReader
from deeppavlov.core.data.data_learning_iterator import DataLearningIterator
from deeppavlov.core.data.utils import download_decompress

In [10]:
# Read FAQ data
reader = FaqDatasetReader()
faq_data = reader.read(data_url=FAQ_DATASET_URL, x_col_name='Question', y_col_name='Answer')
iterator = DataLearningIterator(data=faq_data)

x,y = iterator.get_instances()

## Train FAQ

Let's consider simple case for FAQ model (in the end you can find more complex pipeline models):
1. TF_IDF vectorizer on lemmatized questions
2. Logistic regression classifier

In [11]:
# create tokenizer
tokenizer = StreamSpacyTokenizer(lemmas=True)
x_tokenized = tokenizer(x)
# fit TF-IDF vectorizer on train FAQ dataset 
vectorizer = TfIdfVectorizer(mode='train')
vectorizer.fit(x_tokenized)

# Now collect (x,y) pairs: x_train - vectorized question, y_train - answer from FAQ
x_train = vectorizer(x_tokenized)
y_train = y 

# Let's use top 2 answers for each incoming questions (top_n param)
clf= LogregClassifier(mode='train', top_n=2, c=1000, penalty='l2', save_path='faq/tfidf_logreg_classifier_en_mipt_faq.pkl', load_path='faq/tfidf_logreg_classifier_en_mipt_faq.pkl')
clf.fit(x_train, y_train)


## Test FAQ

In [12]:
test_questions = ['Could you help me??', 'Could I work while studying?']
tokenized_test_questions = tokenizer(test_questions)
test_q_vectorized = vectorizer(tokenized_test_questions)
answers = clf(test_q_vectorized)

Now we have all output of FAQ model: answers and scores.
<br>
Answers:

In [13]:
for i, answer in enumerate(answers[0]):
    print('Answers {}:\n{}\n'.format(i, answer))

Answers 0:
['If you have any problems you can address to Department of Foreign Students: +7 (495) 408-70-43 (Auditorium building, room 315).', 'Life insurance and health is obligatory for any foreign citizen who arrived to Russian Federation for study.']

Answers 1:
['Russian education is one of the most qualitative and fundamental in the world. It allows the student to find well paid work and to start climbing up on a career ladder right after completing university course. Students of the Russian universities are obliged to attend all lectures as only the knowledge gained during classroom occupations allows students to become the effective and knowing professionals. Thus, there is an opportunity to work only after classes or during vacation on the weekend.', "Russian taught programs cost 250'000 rubles per year, English taught — 400'000 rubles per year."]



Let's see score for each answer (score: [0,1])

In [14]:
for i, score in enumerate(answers[1]):
    print('Scores {}:{}'.format(i, score))

Scores 0:[0.93, 0.01]
Scores 1:[0.87, 0.06]


## More models

Described model built in config - deeppavlov/configs/faq/tfidf_logreg_en_faq.json

You can also combine different components to construct pipelines for FAQ task:

Vectorizers:
 - deeppavlov.core.models.vectorizers.TfIdfVectorizer
 - deeppavlov.core.models.vectorizers.SentenceAvgW2vVectorizer
 - deeppavlov.core.models.vectorizers.SentenceW2vVectorizerTfidfWeights

Classifiers:
 - deeppavlov.models.classifiers.logreg_classifier.LogregClassifier
 - deeppavlov.models.classifiers.cos_sim_classifier.CosineSimilarityClassifier

